<a href="https://colab.research.google.com/github/profteachkids/chetools/blob/main/least_squares_linear_operator_jax.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from scipy.optimize import least_squares, root
import jax.numpy as jnp
import jax
import numpy as np
from scipy.sparse.linalg import LinearOperator
from jax.config import config
from functools import partial
config.update("jax_enable_x64", True)

In [66]:
N=1000
N_offdiagonal = int(10*N)
xguess=jnp.asarray(np.random.uniform(size=N))
pairs = np.random.randint(0,N,size=(N_offdiagonal,2))
eq = np.random.randint(0,N,N_offdiagonal)
coeff = np.random.uniform(-10,10,size=N)

In [67]:
def func(x):
    x=jnp.asarray(x)+coeff
    res=x**2
    res=res.at[eq].add(-x[pairs[:,0]]*x[pairs[:,1]])
    return res
jit_func = jax.jit(func)

In [68]:
def mv(x,v):
   return jax.jvp(func,(x,),(jnp.squeeze(v),))[1]

jit_mv = jax.jit(mv)

In [69]:
def rmv(x,v):
    return jax.vjp(func,x)[1](v)

jit_rmv = jax.jit(rmv)

In [70]:
def LO(x):
    return LinearOperator((N,N), matvec = partial(jit_mv,x), rmatvec=partial(jit_rmv, x))

In [71]:
sol=least_squares(jit_func,xguess,jac=LO,gtol=1e-12)

In [ ]:
func(sol.x)

In [38]:
sol.x

array([-6.33352467, -0.61726225, -4.40946944, ..., -8.06705956,
       -0.54791887, -1.51400014])

In [39]:
sol.nfev, sol.njev

(21, 21)